# Understanding Invertible Logic Gates

p-Bits can be understood as a Bernoulli variable parameterized by a probability $p$ - ie. when a p-bit is measured a proportion $p$ of the time it will be $1$ and $1-p$ of the time it will be a $0$. We usually let this $p$ parameter be controlled by some external source which can turn up or turn down $p$.

In this way, multiple p-bits can be connected together to form small Boltzmann machines. Boltzmann machines can be thought of as neural nets which recursively propagate a stochastic signal through itself. If we want to find particularly useful functions of these Boltzmann machines, we can use linear programming that finds weight and bias terms that are congruent with a desired function. To do this we first define the energy of a given Boltzmann machine, in terms of its biases and p-bit output it given by the following equation:

$$
\begin{equation*} H = - \sum _{i} h_{i} m_{i} - \sum _{i < j} J_{ij} m_{i} m_{j}. \tag{1}\end{equation*}
$$

Where $m_i$ is the output of the $i^{\text{th}}$ p-bit and $J_{ij}$ is the weight between and $h_i$ is the bias term. We fix the energy profile of the behaviour that we would like to see, and then we use linear programming to find the $J_{ij}$ and $h_{i}$ that create this energy profile given a certain input.

For example, if we were trying to define a Boltzmann AND-gate (in p-computing this called the invertible AND), we'd define the following energy profile:

A | B | C = AND(A,B)| Valid? | H (Energy) |
--|---|---|--------|---|
-1 | -1 | -1 |Yes|$E_0 = E_{min}$|
-1 | +1 | -1 |Yes|$E_1 = E_{min}$|
+1 | -1 | -1 |Yes|$E_2 = E_{min}$|
+1 | +1 | -1 |No|$E_3 \geq E_{min} + d$|
-1 | -1 | +1 |No|$E_4 \geq E_{min} + d$|
-1 | +1 | +1 |No|$E_5 \geq E_{min} + d$|
+1 | -1 | +1 |No|$E_6 \geq E_{min} + d$|
+1 | +1 | +1 |Yes|$E_7 = E_{min}$|

For each state, we describe the energy that we would like to see from our Boltzmann machine and then we use Linear Programming to find the weights and biases necessary to create this energy profile.

In [66]:
import pulp

problem = pulp.LpProblem('AND',pulp.LpMinimize)

# Energy
E = pulp.LpVariable('E',-10,10,'Continuous')
# Biases
h = pulp.LpVariable.dicts('h',([0],[0,1,2]),-5,5,'Integer')
# Weights
j = pulp.LpVariable.dicts('j',([0],[0,1,2]),-5,5,'Integer')

# Dummy summation variables
H_h, H_j = (0,0)

# Iterate over rows of the above table
for i in range(8):
    # Generate correct values
    A = 2 * int(i % 4 >= 2) - 1
    B = 2 * (i % 2) - 1
    C = 2 * int(i >= 4) - 1

    # Individual terms of final energy
    H_hp = A*h[0][0] + B*h[0][1] + C*h[0][2]
    H_jp = A*B*j[0][0] + B*C*j[0][1] + A*C*j[0][2]

    # Summed terms of final energy
    H_h = H_h + H_hp
    H_j = H_j + H_jp

    # Add energy constraints determined above
    if (A > 0) & (B > 0) == (C > 0):
        problem += (-1)*H_hp-H_jp-E == 0
    else:
        problem += (-1)*H_hp-H_jp-E-1 >= 0

# Finally layout the objective function to minimize
problem += -H_h-H_j-E

# Check that problem described is what was intended
problem.coefficients

<bound method LpProblem.coefficients of AND:
MINIMIZE
-1*E + 0*h_0_0 + 0*h_0_1 + 0*h_0_2 + 0*j_0_0 + 0*j_0_1 + 0*j_0_2 + 0
SUBJECT TO
_C1: - E + h_0_0 + h_0_1 + h_0_2 - j_0_0 - j_0_1 - j_0_2 = 0

_C2: - E + h_0_0 - h_0_1 + h_0_2 + j_0_0 + j_0_1 - j_0_2 = 0

_C3: - E - h_0_0 + h_0_1 + h_0_2 + j_0_0 - j_0_1 + j_0_2 = 0

_C4: - E - h_0_0 - h_0_1 + h_0_2 - j_0_0 + j_0_1 + j_0_2 >= 1

_C5: - E + h_0_0 + h_0_1 - h_0_2 - j_0_0 + j_0_1 + j_0_2 >= 1

_C6: - E + h_0_0 - h_0_1 - h_0_2 + j_0_0 - j_0_1 + j_0_2 >= 1

_C7: - E - h_0_0 + h_0_1 - h_0_2 + j_0_0 + j_0_1 - j_0_2 >= 1

_C8: - E - h_0_0 - h_0_1 - h_0_2 - j_0_0 - j_0_1 - j_0_2 = 0

VARIABLES
-10 <= E <= 10 Continuous
-5 <= h_0_0 <= 5 Integer
-5 <= h_0_1 <= 5 Integer
-5 <= h_0_2 <= 5 Integer
-5 <= j_0_0 <= 5 Integer
-5 <= j_0_1 <= 5 Integer
-5 <= j_0_2 <= 5 Integer
>

In [67]:
# Solve problem in question
problem.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/thomas/.local/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/c97a3ec3c5464370b511003a8a6427a4-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/c97a3ec3c5464370b511003a8a6427a4-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 13 COLUMNS
At line 89 RHS
At line 98 BOUNDS
At line 113 ENDATA
Problem MODEL has 8 rows, 7 columns and 56 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0.75 - 0.00 seconds
Cgl0004I processed model has 8 rows, 7 columns (6 integer (0 of which binary)) and 56 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0012I Integer solution of 3 found by DiveCoefficient after 5 iterations and 0 nodes (0.00 seconds)
Cbc0031I 1 added rows had average density of 7
Cbc0013I At root node, 1 cuts changed objective from 0.75 to 2.99999

1

In [68]:
# Get final values of output
print("J_AB=",pulp.value(j[0][0]))
print("J_BC=",pulp.value(j[0][1]))
print("J_AC=",pulp.value(j[0][2]))
print("- "*5)
print("h_A=",pulp.value(h[0][0]))
print("h_B=",pulp.value(h[0][1]))
print("h_C=",pulp.value(h[0][2]))

J_AB= -1.0
J_BC= 2.0
J_AC= 2.0
- - - - - 
h_A= 1.0
h_B= 1.0
h_C= -2.0


This gives us the weights and biases that we desire, and gives us the final Hamiltonian that we seek:

$$
J_{AND} = 
\begin{bmatrix}
J_{AA} & J_{AB} & J_{AC} \\
J_{BA} & J_{BB} & J_{BC} \\
J_{CA} & J_{CB} & J_{CC} 
\end{bmatrix} =
\begin{bmatrix}
0 & -1 & +2 \\
-1 & 0 & +2 \\
+2 & +2 & 0 
\end{bmatrix}
\qquad
h = \begin{bmatrix}
+1 & +1 & -2
\end{bmatrix}
$$

This acts as our update rule across the Boltzmann Machine, and with it we can do a preliminary simulation of p-bit computation and see exactly why what we've found is actually an "invertible" logic gate.

In [167]:
import numpy as np
from objects import pbit

# Here we first describe the general p-bit network
class pbit_network:
    def __init__(self, pbits: list, J: np.array, h: np.array):

        self.pbits = pbits
        self.J = J
        self.h = h

    def step(self, dt = 1):

        sample = [p.sample() for p in self.pbits]
        forward = (self.J * dt) @ np.array(sample).T + self.h * dt

        for idx, p in enumerate(self.pbits):
            p.activation = forward[idx]

    def sample(self):
        return [p.sample() for p in self.pbits]


class AND(pbit_network):

    def __init__(self):

        # Note in this format we take the first two pbits to be inputs A,B
        # and the final pbit to be the output C = AND(A,B)
        J = np.array([[0, -1, 2], [-1, 0, 2], [2, 2, 0]])
        h = np.array([1, 1, -2])
        super().__init__([pbit(), pbit(), pbit()], J, h)

We'll now run two experiments to see exactly what we mean by an invertible logic gate:

##### Experiment 1 - Ordinary Logic Gate

We should expect that if we clamp A and B to different values that we get the correct result, ie. C = AND(A,B).

##### Experiment 2 - Invertibility

Our logic gate is invertible, that means that if we clamp C to some output - statistically, A and B will converge to values such that AND(A,B) will be the C we clamped.

In [114]:
# Experiment 1

print("- - - EXPERIMENT 1 - - -\n")

num_steps = 10
num_samples = 100

# Iterate over possible inputs
for i in range(4):

    and_gate = AND()

    A = 2 * (i % 2) - 1
    B = 2 * (i // 2) - 1

    and_gate.pbits[0].clamped = A
    and_gate.pbits[1].clamped = B

    # Let it converge
    for i in range(num_steps):
        and_gate.step()

    # Take some samples
    samples = []
    for i in range(num_samples):
        samples.append(and_gate.sample())

    count = sum([s[2] > 0 for s in samples])

    print(f"{(A+1)//2} AND {(B+1)//2} = {count/num_samples}")

- - - EXPERIMENT 1 - - -

0 AND 0 = 0.0
1 AND 0 = 0.02
0 AND 1 = 0.0
1 AND 1 = 0.97


In [139]:
# Experiment 2

num_steps = 10
num_samples = 100

print("- - - EXPERIMENT 2 - - -\n")

# Iterate over possible inputs
for i in range(2):

    and_gate = AND()

    C = 2 * i - 1

    and_gate.pbits[2].clamped = C

    # Let it converge
    for i in range(num_steps):
        and_gate.step()

    # Take some samples
    samples = []
    for i in range(num_samples):
        samples.append(and_gate.sample())

    countA = sum([(s[0] > 0) & (s[1] > 0)for s in samples])

    print(f"C={(C+1)//2} implies AND(A,B)={countA/num_samples}")

- - - EXPERIMENT 2 - - -

C=0 implies AND(A,B)=0.01
C=1 implies AND(A,B)=0.97


So we can clearly see the reversibility in action. We'll conduct one final experiment to precisely explain the exact nature of this invertibility:

##### Experiment 3 - Inversion Distribution

If AND(A,B) = 0, there are 3 possible combinations

In [160]:
num_steps = 10
num_samples = 100

and_gate = AND()

C = -1

and_gate.pbits[2].clamped = C

# Let it converge
for i in range(num_steps):
    and_gate.step()

# Take some samples
samples = []
for i in range(num_samples):
    samples.append(and_gate.sample())

count00 = sum([(s[0] < 0) & (s[1] < 0)for s in samples])
count01 = sum([(s[0] < 0) & (s[1] > 0)for s in samples])
count10 = sum([(s[0] > 0) & (s[1] < 0)for s in samples])
count11 = sum([(s[0] > 0) & (s[1] > 0)for s in samples])

print(f"C={(C+1)//2} implies\n\nP(0&0)={count00/num_samples}\nP(0&1)={count01/num_samples}\nP(1&0)={count10/num_samples}\nP(1&1)={count11/num_samples}")

C=0 implies

P(0&0)=0.27
P(0&1)=0.27
P(1&0)=0.26
P(1&1)=0.2


Depending on the random seed, the network has a tendency to converge to different distributions of "the most likely answer". This appears to be fairly random in how it converges - let's see if an average begins to emerge:

##### Experiment 4 - Inversion Statistics

We explore the final statistics more closely.

In [174]:
from numpy import mean, std

p00 = []
p01 = []
p10 = []
p11 = []

num_steps = 1000
num_samples = 100

for i in range(1000):
    and_gate = AND()

    C = -1

    and_gate.pbits[2].clamped = C

    # Let it converge
    for i in range(num_steps):
        and_gate.step(dt=1)

    # Take some samples
    samples = []
    for i in range(num_samples):
        samples.append(and_gate.sample())

    p01.append(sum([(s[0] < 0) & (s[1] > 0)for s in samples])/num_samples)
    p10.append(sum([(s[0] > 0) & (s[1] < 0)for s in samples])/num_samples)
    p00.append(sum([(s[0] < 0) & (s[1] < 0)for s in samples])/num_samples)
    p11.append(sum([(s[0] > 0) & (s[1] > 0)for s in samples])/num_samples)

print(f"C={(C+1)//2} implies\n\nP(0&0) mean:{mean(p00)}, std:{std(p00)}")
print(f"P(0&1) mean:{mean(p01)}, std:{std(p01)}")
print(f"P(1&0) mean:{mean(p10)}, std:{std(p10)}")
print(f"P(1&1) mean:{mean(p11)}, std:{std(p11)}")

C=0 implies

P(0&0) mean:0.44537, std:0.22902852027640572
P(0&1) mean:0.21835000000000002, std:0.1793744059223612
P(1&0) mean:0.22303, std:0.18073217505469247
P(1&1) mean:0.11325, std:0.12345257186466388
